In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#데이터읽기
customers = pd.read_csv('./dacon_data/open/customers.csv')
locations = pd.read_csv('./dacon_data/open/locations.csv')
orders = pd.read_csv('./dacon_data/open/orders.csv')
order_items = pd.read_csv('./dacon_data/open/order_items.csv')
payments = pd.read_csv('./dacon_data/open/payments.csv')
products = pd.read_csv('./dacon_data/open/products.csv')
reviews = pd.read_csv('./dacon_data/open/reviews.csv')
sellers = pd.read_csv('./dacon_data/open/sellers.csv')

In [51]:
# 데이터 merge하기
temp = pd.merge(reviews,order_items, how='inner',on='Order_id')
temp = pd.merge(temp,orders, how='inner',on='Order_id')
temp = pd.merge(temp,payments, how='inner',on='Order_id')
temp = pd.merge(temp,products, how='inner',on='Product_id')
temp = pd.merge(temp,customers, how='inner',on='Customer_id')
temp = pd.merge(temp,sellers, how='inner',on='Seller_id')

In [52]:
# locations[locations.Geolocation_zipcode_prefix==1037]
# 1037을 가진 우편번호가 여러개 존재함

In [53]:
# 여러개의 우편번호가 존재하므로 한우편번호에 있어서 평균을 구하기로함
locations_group_lat=locations.groupby(["Geolocation_zipcode_prefix"])["Geolocation_lat"].mean().reset_index()
locations_group_lng=locations.groupby(["Geolocation_zipcode_prefix"])["Geolocation_lng"].mean().reset_index()
locations_group =  pd.merge(locations_group_lat, locations_group_lng, how = 'inner', on="Geolocation_zipcode_prefix")

In [54]:
# 구입자의 위도경도
temp = pd.merge(temp, locations_group, left_on = 'Customer_zipcode_prefix', right_on = 'Geolocation_zipcode_prefix', how = 'inner')
temp = temp.rename(columns={'Geolocation_lat':'Customer_lat',"Geolocation_lng" : "Customer_lng" })

In [55]:
# 판매자의 위도경도
temp = pd.merge(temp, locations_group, left_on = 'Seller_zipcode_prefix', right_on = 'Geolocation_zipcode_prefix', how = 'inner')
temp = temp.rename(columns={'Geolocation_lat':'Seller_lat',"Geolocation_lng" : "Seller_lng" })

In [56]:
# 중복되는 구매자,판매자 우편번호 드랍
temp = temp.drop(["Geolocation_zipcode_prefix_x","Geolocation_zipcode_prefix_y"],axis=1)

In [57]:
# date 타입으로 변환
temp['Order_purchase_timestamp']=pd.to_datetime(temp['Order_purchase_timestamp'])
temp['Order_delivered_carrier_date']=pd.to_datetime(temp['Order_delivered_carrier_date'])
temp['Order_delivered_customer_date']=pd.to_datetime(temp['Order_delivered_customer_date'])
temp['Order_estimated_delivery_date']=pd.to_datetime(temp['Order_estimated_delivery_date'])
# temp[['Order_delivered_carrier_date','Order_delivered_customer_date',"Order_purchase_timestamp"]] = temp[['Order_delivered_carrier_date','Order_delivered_customer_date',"Order_purchase_timestamp"]].apply(pd.to_datetime)

In [58]:
# temp의 시계열이 datetype으로 바뀌었는지 확인
# temp.info()

In [59]:
# Order_purchase_timestamp : 구매 시간
# Order_delivered_carrier_date : 물류 처리 시간
# Order_delivered_customer_date : 실제 배송 날짜
# Order_estimated_delivery_date : 기대 배송 날짜

# 구매할때부터 물류처리할때까지 걸리는 시간
temp["time_diff_purchase_carrier"] = temp["Order_delivered_carrier_date"] - temp["Order_purchase_timestamp"]
# 구매할때부터 실제배송할때까지 걸리는 시간
temp["time_diff_purchase_customer"] = temp["Order_delivered_customer_date"] - temp["Order_purchase_timestamp"]
# 물류처리할때부터 실제배송할때까지 걸리는 시간
temp["time_diff_carrier_customer"] = temp["Order_delivered_customer_date"] - temp["Order_delivered_carrier_date"]
# 기대배송날짜와 실제배송날짜의 차이
temp["time_diff_customer_delivery"]= temp['Order_estimated_delivery_date'] - temp["Order_delivered_carrier_date"]

In [60]:
# 상관관계를 구하기 위하여 2days xx:xx:xx을 초로 변환
temp["time_diff_purchase_carrier_second"] = temp["time_diff_purchase_carrier"].apply(lambda x : x.total_seconds())
temp["time_diff_purchase_customer_second"] = temp["time_diff_purchase_customer"].apply(lambda x : x.total_seconds())
temp["time_diff_carrier_customer_second"] = temp["time_diff_carrier_customer"].apply(lambda x : x.total_seconds())
temp["time_diff_customer_delivery_second"] = temp["time_diff_customer_delivery"].apply(lambda x : x.total_seconds())

In [61]:
# 구매자와 판매자의 거기 변수 생성
temp["Distance"] = ((temp["Customer_lat"] - temp["Seller_lat"])**2 + (temp["Customer_lng"] - temp["Seller_lng"])**2) **(1/2)

In [62]:
# 일단 매출액과 기간의 상관관계를 보아보자
temp_corr = temp[["Review_score","Price","Freight_value","Payment_value","Product_weight_g","time_diff_purchase_carrier_second","time_diff_purchase_customer_second","time_diff_carrier_customer_second","time_diff_customer_delivery_second","Distance"]]

In [63]:
# 원본데이터일때 상관관계
temp_corr.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)

,Review_score,Price,Freight_value,Payment_value,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second,Distance
Review_score,1.000000,0.004214,-0.035906,-0.082358,-0.149300,-0.308265,-0.275310,0.008873,-0.058759
Price,0.004214,1.000000,0.410670,0.732582,0.063354,0.061012,0.040461,0.043452,0.084659
Freight_value,-0.035906,0.410670,1.000000,0.368607,0.093304,0.220326,0.202336,0.251835,0.389278
Payment_value,-0.082358,0.732582,0.368607,1.000000,0.095513,0.059390,0.025340,0.043680,0.086384
time_diff_purchase_carrier_second,-0.149300,0.063354,0.093304,0.095513,1.000000,0.406626,0.029893,-0.217128,0.019257
time_diff_purchase_customer_second,-0.308265,0.061012,0.220326,0.059390,0.406626,1.000000,0.925342,0.196048,0.399764
time_diff_carrier_customer_second,-0.275310,0.040461,0.202336,0.025340,0.029893,0.925342,1.000000,0.304600,0.429385
time_diff_customer_delivery_second,0.008873,0.043452,0.251835,0.043680,-0.217128,0.196048,0.304600,1.000000,0.512659
Distance,-0.058759,0.084659,0.389278,0.086384,0.019257,0.399764,0.429385,0.512659,1.000000


In [64]:
# orders

In [65]:
# 제대로 sum이 안되는 columns 확인
# Product_weight_g 하나뿐 object로 되어있다.
# 타입 변경해주기
temp['Product_weight_g'] = temp['Product_weight_g'].replace('Unknown', np.nan).fillna(0).astype(float)

In [66]:
# 배송비를 줄이면 리뷰점수가 오른다.
# 결제금액이 클수록 만족도가 떠러진다.
# 리뷰점수가 좋으면 판매자가 인기가 있는가??

# merge된 temp를 주별로 묶기
temp_group_city = temp.groupby(["Customer_state"])[["Review_score","Price","Freight_value","Payment_value","Product_weight_g","Distance","time_diff_purchase_carrier_second","time_diff_purchase_customer_second","time_diff_carrier_customer_second","time_diff_customer_delivery_second"]].mean().reset_index()

In [104]:
payments
# 할부가 존재하는건 신용카드

,Order_id,Payment_sequential,Payment_type,Payment_installments,Payment_value
0,ORDER_66617,1,credit_card,8,99.33
1,ORDER_86954,1,credit_card,1,24.39
2,ORDER_38852,1,credit_card,1,65.71
3,ORDER_57443,1,credit_card,8,107.78
4,ORDER_11244,1,credit_card,2,128.45
...,...,...,...,...,...
91966,ORDER_46197,1,credit_card,1,194.11
91967,ORDER_37509,1,credit_card,2,198.94
91968,ORDER_85760,1,boleto,1,363.31
91969,ORDER_35237,1,credit_card,2,96.80


In [67]:
temp_group_city.head(1)
#303591.691358 이게 구매할때부터 포장할때까지의 ㅅ간평균이라고?

,Customer_state,Review_score,Price,Freight_value,Payment_value,Product_weight_g,Distance,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second
0,AC,4.012346,183.849383,40.437531,235.288889,2057.320988,24.502951,303591.691358,1.871732e+06,1.568140e+06,3.303366e+06


In [68]:
# 주로 mean한 상관관계 구하기
# temp_group_city.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)

In [69]:
# 일단 기대배송일자와 매출액을 비교해봤을때 
# time_diff_customer_delivery_second와 Price
# 0.547777 양의 상관관계를 가지고있다. 
# 위의 표를 정리해보면 0.4 이상을 유의미한 값으로 지정

# 반비례 : 리뷰점수와 구매시간부터 포장시간 차이(-0.48), 
        
# 비례   : 순이익과 운임요금(0.86), 순이익과 거래가치(0.85), 가격과 거리(0.71), 가격과 구매부터 도착까지(0.65), 가격과 포장부터 도착(0.64)
        #  가격과 기대배송날짜와 실제배송날짜의 차이(0.54), 운임요금과 거래가치(0.83), 운임요금과 거리(0.88), 운인요금과 구매부터 도착(0.77)
        # 운임요금과 포장부터 도착(0.76), 운임요금과 ㄱ대배송 실제배송창(0.70), 거래가치는 거리(0.73), 거래가치와 구입부터 배송까지(0.74),
        # 거래가치와 포장부터 고객까지(0.73), 거래가치와 실제배송 예측차이(0.62), 거리와 구입부터 고객까지(0.91), 거리와 포장고객(0.91),
        # 거리와 실제배송 예측(0.89), 

In [70]:
#주에 따라 묶어서 주에서 거래한 개수
temp_group_count= temp.groupby(["Customer_state"])["Customer_city"].count().reset_index()
temp_group_count["count_ratio"]=temp_group_count["Customer_city"].apply(lambda x : x/temp.shape[0])

In [71]:
# 그걸 기존 df와 묶기
temp_group_city= pd.merge(temp_group_city, temp_group_count, on='Customer_state', how='inner')

In [72]:
# 전체 순수익 더하기
all_price_sum=temp.Price.sum()

In [73]:
# 주에 따라 묶어서 순수익률 개수
temp_group_price= temp.groupby(["Customer_state"])["Price"].sum().reset_index()
temp_group_price["price_ratio"]=temp_group_price["Price"].apply(lambda x : x/all_price_sum)

In [74]:
# 그걸 기존 df와 묶기
temp_group_city= pd.merge(temp_group_city, temp_group_price, on='Customer_state', how='inner')

In [75]:
# 중복도는 컬럼 drop
temp_group_city = temp_group_city.drop(["Price_y"],axis=1)

In [76]:
# # 실질적인 ratio 비율 확인
# temp_group_city["ratio_gap"] = temp_group_city["price_ratio"] - temp_group_city["count_ratio"]

In [77]:
# 전체 매출액 더하기
all_payment_sum=temp.Payment_value.sum()

In [78]:
#주에 따라 묶어서 주에서 매출액
temp_group_payment= temp.groupby(["Customer_state"])["Payment_value"].sum().reset_index()
temp_group_payment["payment_ratio"]=temp_group_payment["Payment_value"].apply(lambda x : x/all_payment_sum)

In [79]:
# 그걸 기존 df와 묶기
temp_group_city= pd.merge(temp_group_city, temp_group_payment, on='Customer_state', how='inner')

In [80]:
# 중복도는 컬럼 drop
temp_group_city = temp_group_city.drop(["Payment_value_y"],axis=1)

In [81]:
#count_ratio 가 낮으면서 price_ratio 가 높은게 가치가 있다. 파생변수 생성
temp_group_city["value_ratio"] = temp_group_city["price_ratio"] / temp_group_city["count_ratio"] #짜피 payment_ratio와 연관성이 85

In [82]:
# value_ratio 기준으로 정렬하기
temp_group_city_sorted= temp_group_city.sort_values(["value_ratio"],ascending=False)

In [83]:
# 1이 안되는 애들은 판매되는 개수는 많은데 순이익이 안나오므로 그에 비하여 성장시킬 필요가 없다.(물론 손해를 안보는 쪽으로 하는)
temp_group_city_sorted_up = temp_group_city_sorted[temp_group_city_sorted.value_ratio>1]

In [84]:
# 1이 안되는 애들은 판매되는 개수는 많은데 순이익이 안나오므로 그에 비하여 성장시킬 필요가 없다.(물론 손해를 안보는 쪽으로 하는)
temp_group_city_sorted_down = temp_group_city_sorted[temp_group_city_sorted.value_ratio<=1]

In [85]:
# 가치가 있는 애들의 상관관계 구하기
temp_group_city_sorted_up.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)
# 해석
# 물건살때부터 포장할때까지 걸리는 시간이 줄어들면 리뷰점수가올라간다. -0.48이므로 100이 오르면 48이 바뀐다.

,Review_score,Price_x,Freight_value,Payment_value_x,Product_weight_g,Distance,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second,Customer_city,count_ratio,price_ratio,payment_ratio,value_ratio
Review_score,1.000000,0.011660,-0.131562,-0.062691,0.080787,-0.085716,-0.482678,-0.151813,-0.121068,0.180405,-0.155730,-0.155730,-0.171888,-0.179001,0.011660
Price_x,0.011660,1.000000,0.808580,0.797183,0.260325,0.598075,0.337138,0.489021,0.471009,0.379988,-0.568783,-0.568783,-0.555846,-0.555814,1.000000
Freight_value,-0.131562,0.808580,1.000000,0.758344,0.211247,0.833370,0.308626,0.658513,0.643931,0.590222,-0.631293,-0.631293,-0.624273,-0.620833,0.808580
Payment_value_x,-0.062691,0.797183,0.758344,1.000000,0.073847,0.634039,0.256010,0.632744,0.621430,0.484672,-0.526559,-0.526559,-0.515181,-0.504017,0.797183
Product_weight_g,0.080787,0.260325,0.211247,0.073847,1.000000,-0.242614,0.080933,-0.305535,-0.313683,-0.302285,0.033628,0.033628,0.038388,0.040231,0.260325
Distance,-0.085716,0.598075,0.833370,0.634039,-0.242614,1.000000,0.173061,0.878400,0.874827,0.863615,-0.619594,-0.619594,-0.616069,-0.614583,0.598075
time_diff_purchase_carrier_second,-0.482678,0.337138,0.308626,0.256010,0.080933,0.173061,1.000000,0.168246,0.103227,-0.131581,-0.031034,-0.031034,-0.022103,-0.022014,0.337138
time_diff_purchase_customer_second,-0.151813,0.489021,0.658513,0.632744,-0.305535,0.878400,0.168246,1.000000,0.997847,0.858570,-0.484725,-0.484725,-0.483130,-0.478341,0.489021
time_diff_carrier_customer_second,-0.121068,0.471009,0.643931,0.621430,-0.313683,0.874827,0.103227,0.997847,1.000000,0.875089,-0.487043,-0.487043,-0.486027,-0.481201,0.471009
time_diff_customer_delivery_second,0.180405,0.379988,0.590222,0.484672,-0.302285,0.863615,-0.131581,0.858570,0.875089,1.000000,-0.434036,-0.434036,-0.442473,-0.440127,0.379988


In [86]:
import numpy as np
np.random.seed(2021)
X = np.array(temp_group_city_sorted_up["time_diff_purchase_carrier_second"].tolist())
y = np.array(temp_group_city_sorted_up["Review_score"].tolist())
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X=X.reshape(-1, 1), y=y) 

LinearRegression()

In [87]:
model.intercept_ #model.coef_ #[0.6]

4.780350742124

In [88]:
model.coef_ #array([-2.75162645e-06])

array([-2.75162645e-06])

In [89]:
temp_group_city_sorted_up["time_diff_purchase_carrier_second"] = temp_group_city_sorted_up["time_diff_purchase_carrier_second"].apply(lambda x : x-145794)

/tmp/ipykernel_1229/3836647404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_group_city_sorted_up["time_diff_purchase_carrier_second"] = temp_group_city_sorted_up["time_diff_purchase_carrier_second"].apply(lambda x : x-145794)


In [90]:
import numpy as np
np.random.seed(2021)
X = np.array(temp_group_city_sorted_up["time_diff_purchase_carrier_second"].tolist())
y = np.array(temp_group_city_sorted_up["Review_score"].tolist())
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X=X.reshape(-1, 1), y=y) 

LinearRegression()

In [91]:
model.intercept_ #4.37918011476355

4.37918011476355

In [92]:
model.coef_ #array([-2.75162645e-06])

array([-2.75162645e-06])

In [93]:
# 4.780350742124/4.37918011476355 == 1.0916086155049851
# 물품준비시간을 50퍼를 줄이면 10퍼의 리뷰점수가 오르는 효과를 볼수있다. 그런데 

In [94]:
# 가치가 없는 애들의 상관관계 구하기
temp_group_city_sorted_down.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)
# 해석
# 리뷰점수와 운임요금은 반비례다 리뷰점수가 높으면 운임요금이 싸다

,Review_score,Price_x,Freight_value,Payment_value_x,Product_weight_g,Distance,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second,Customer_city,count_ratio,price_ratio,payment_ratio,value_ratio
Review_score,1.000000,-0.278888,-0.773513,-0.305664,0.040566,-0.975277,-0.885743,-0.965299,-0.965822,-0.686430,0.734460,0.734460,0.743277,0.749520,-0.278888
Price_x,-0.278888,1.000000,0.816148,0.897146,0.564644,0.483217,0.412993,0.505600,0.507675,0.878012,-0.811254,-0.811254,-0.797910,-0.793756,1.000000
Freight_value,-0.773513,0.816148,1.000000,0.810469,0.260696,0.894125,0.837886,0.909623,0.910002,0.991758,-0.988726,-0.988726,-0.986774,-0.987406,0.816148
Payment_value_x,-0.305664,0.897146,0.810469,1.000000,0.144674,0.493414,0.605522,0.541361,0.537806,0.872479,-0.867252,-0.867252,-0.862437,-0.857643,0.897146
Product_weight_g,0.040566,0.564644,0.260696,0.144674,1.000000,0.075291,-0.276554,0.038781,0.049843,0.294704,-0.150260,-0.150260,-0.128348,-0.127174,0.564644
Distance,-0.975277,0.483217,0.894125,0.493414,0.075291,1.000000,0.911057,0.997438,0.998126,0.829550,-0.860504,-0.860504,-0.865795,-0.870460,0.483217
time_diff_purchase_carrier_second,-0.885743,0.412993,0.837886,0.605522,-0.276554,0.911057,1.000000,0.935119,0.930605,0.788833,-0.866780,-0.866780,-0.878044,-0.881025,0.412993
time_diff_purchase_customer_second,-0.965299,0.505600,0.909623,0.541361,0.038781,0.997438,0.935119,1.000000,0.999921,0.850191,-0.886096,-0.886096,-0.891831,-0.896045,0.505600
time_diff_carrier_customer_second,-0.965822,0.507675,0.910002,0.537806,0.049843,0.998126,0.930605,0.999921,1.000000,0.850343,-0.884676,-0.884676,-0.890202,-0.894450,0.507675
time_diff_customer_delivery_second,-0.686430,0.878012,0.991758,0.872479,0.294704,0.829550,0.788833,0.850191,0.850343,1.000000,-0.988674,-0.988674,-0.984904,-0.984342,0.878012


In [95]:
temp_group_city_sorted_down.Customer_state

10    MG
17    PR
7     ES
25    SP
Name: Customer_state, dtype: object

In [96]:
# temp_group_city_sorted_down

In [97]:
temp_group_city_sorted

,Customer_state,Review_score,Price_x,Freight_value,Payment_value_x,Product_weight_g,Distance,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second,Customer_city,count_ratio,price_ratio,payment_ratio,value_ratio
1,AL,3.795567,184.119754,35.245813,246.528867,1950.679803,17.024909,319321.546798,2.160449e+06,1.841127e+06,2.510456e+06,406,0.003888,0.005981,0.005598,1.538519
0,AC,4.012346,183.849383,40.437531,235.288889,2057.320988,24.502951,303591.691358,1.871732e+06,1.568140e+06,3.303366e+06,81,0.000776,0.001192,0.001066,1.536259
14,PB,4.023508,182.902315,42.803725,281.165045,2281.244123,19.298416,318265.875226,1.817279e+06,1.499013e+06,2.587447e+06,553,0.005295,0.008093,0.008696,1.528346
26,TO,4.120000,171.019767,39.582000,211.071133,3589.853333,13.015428,308919.310000,1.517516e+06,1.208596e+06,2.252126e+06,300,0.002873,0.004105,0.003542,1.429054
16,PI,3.906561,162.413837,38.612028,246.462425,2491.934394,17.645286,279247.602386,1.711384e+06,1.432136e+06,2.354693e+06,503,0.004816,0.006536,0.006934,1.357142
3,AP,4.297297,161.676622,33.661081,264.598649,1871.054054,23.512257,289961.229730,2.430999e+06,2.141038e+06,3.756678e+06,74,0.000709,0.000957,0.001095,1.350982
13,PA,3.804260,160.043479,34.640071,222.986582,2163.530426,20.589619,299794.632860,2.091381e+06,1.791586e+06,2.974306e+06,986,0.009441,0.012626,0.012297,1.337335
19,RN,4.061185,157.806711,34.894761,196.293901,1942.535373,19.080020,324341.780115,1.727412e+06,1.403071e+06,2.534230e+06,523,0.005008,0.006604,0.005742,1.318645
20,RO,4.049808,155.818851,40.956513,223.376475,2480.704981,20.509862,250603.873563,1.736080e+06,1.485476e+06,3.185226e+06,261,0.002499,0.003254,0.003261,1.302034
5,CE,3.864394,153.950210,32.230696,217.432313,1995.771574,19.726289,297688.116026,1.849262e+06,1.551574e+06,2.424422e+06,1379,0.013204,0.016986,0.016770,1.286420
